In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Bidirectional A$^*$ First Search

The module <a href="https://docs.python.org/3.7/library/heapq.html">heapq</a> provides 
<a href="https://en.wikipedia.org/wiki/Priority_queue">priority queues</a> 
that are implemented as 
<a ref="https://en.wikipedia.org/wiki/Heap_(data_structure)">heaps</a>.
Technically, these heaps are just lists.  In order to use them as priority queues, the entries of these lists will be pairs of the form $(p, o)$, where $p$ is the priority of the object $o$.  Usually, the priorities are numbers 
and, contra-intuitively, high priorities correspond to *small* numbers, that is $(p_1, o_1)$ has a higher priority than $(p_2, o_2)$ iff $p_1 < p_2$.

We need only two functions from the module `heapq`:
- Given a heap $H$, the function $\texttt{heapq.heappop}(H)$ removes the pair
  from H that has the highest priority.  This pair is also returned.
- Given a heap $H$, the function $\texttt{heapq.heappush}\bigl(H, (p, o)\bigr)$ 
  pushes the pair $(p, o)$ onto the heap $H$.  This method does not return a 
  value.  Instead, the heap $H$ is changed in place.

In [2]:
import heapq

The function `search` takes three arguments to solve a *search problem*:
- `start` is the start state of the search problem,
- `goal` is the goal state, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
  For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.
- `heuristic` is a function that takes two states as arguments.  It returns an estimate of the 
  length of the shortest path between these states.
If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$

The variable `PrioQueue` that is used in the implementation contains pairs of the form
$$ \bigl(\texttt{len}(\texttt{Path}) + \texttt{heuristic}(\texttt{state},\; \texttt{goal}), \texttt{Path}\bigr), $$
where `Path` is a path from `start` to `state` and $\texttt{heuristic}(\texttt{state}, \texttt{goal})$
is an estimate of the distance from `state` to `goal`.  The idea is to always extend the most promising `Path`, i.e. to extend the `Path` whose completed version would be shortest.

In [3]:
def search(start, goal, next_states, heuristic):
    VisitedA   = {}
    VisitedB   = {}
    PrioQueueA = [ (heuristic(start, goal), [start]) ]
    PrioQueueB = [ (heuristic(goal, start), [goal ]) ]
    while PrioQueueA and PrioQueueB:
        a, PathA = heapq.heappop(PrioQueueA)
        b, PathB = heapq.heappop(PrioQueueB)
        if a <= b:
            heapq.heappush(PrioQueueB, (b, PathB))
            state = PathA[-1]
            if state in VisitedA:
                continue
            if state in VisitedB: 
                return PathA[:-1] + VisitedB[state][::-1]
            for ns in next_states(state):           
                if ns not in VisitedA:
                    prio = len(PathA) + heuristic(ns, goal)
                    heapq.heappush(PrioQueueA, (prio, PathA + [ns]))
            VisitedA[state] = PathA
        else:
            heapq.heappush(PrioQueueA, (a, PathA))
            state = PathB[-1]
            if state in VisitedB:
                continue
            if state in VisitedA:
                return VisitedA[state][:-1] + PathB[::-1]
            for ns in next_states(state):
                if ns not in VisitedB:
                    prio = heuristic(start, ns) + len(PathB)
                    heapq.heappush(PrioQueueB, (prio, PathB + [ns]))
            VisitedB[state] = PathB

In [4]:
%run Sliding-Puzzle.ipynb

In [5]:
%load_ext memory_profiler

In [6]:
%%time
%memit Path = search(start, goal, next_states, manhattan)
print(len(Path)-1)

peak memory: 99.86 MiB, increment: 1.59 MiB
31
CPU times: user 73 ms, sys: 12.9 ms, total: 85.9 ms
Wall time: 581 ms


In [ ]:
animation(Path)

In [7]:
%%time
%memit Path = search(start2, goal2, next_states, manhattan)
print(len(Path)-1)

peak memory: 107.93 MiB, increment: 8.05 MiB
40
CPU times: user 269 ms, sys: 14 ms, total: 283 ms
Wall time: 772 ms


In [ ]:
animation(Path)